In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import seaborn as sns
import numpy as np
import tensorflow as tf
import pandas as pd
import os
from tensorflow import keras
import pymongo

In [2]:
def get_training():
    client = pymongo.MongoClient("mongodb://192.168.99.100:27017")
    db = client['plp']
    a= list(db["match"].find({}))
    a = pd.DataFrame(a)
    a = a.drop(columns = a.columns[0])
    return a

In [3]:
def print_logits (y,batch_size,nb_label):
    # Print a the label predicted
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    for i in range (0,batch_size):
        print("------------",i,"------------")
        for j in range (0,nb_label):
            if (y[i][j] != 0):
                print("prédiction : ",j,"pourcentage de certitude :","%.1f" % (y[i][j]*100) , " %")

def int_to_dummies(i):
    # Create a label for the number given
    #Input: int The value of the label
    result = [0]*3
    result[int(i)] = 1
    return np.array(result)

def create_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
  ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=[
                      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall'),
                      tf.keras.metrics.AUC(name='auc')])

    return model



def prediction_logits (y,batch_size,nb_label):
    # Normalize the batch of label to obtain a percentage
    #Input : 
    #y : Numpy array [batch_size][10] The data predicted
    #batch_size : Int The size of the batch
    #nb_label : Int The number of labels
    #Out : Numpy array [batch_size][10] the input normalized
    
    y_sum = np.sum(y, axis=1)
    for i in range (0,batch_size):
        for j in range (0,nb_label):
            if (y_sum[i] != 0):
                y[i][j] = y[i][j]/y_sum[i]
    
    return y


In [4]:
df = get_training()
df = df[df.ID_FTR != -1]
print(df)
target = df["ID_FTR"].apply(int_to_dummies)
features = df[df.columns[6:]]

      Div        Date               HomeTeam        AwayTeam FTR  ID_FTR  \
0      F1    20/09/14                 Nantes            Nice   H       2   
1      F1    21/09/14                  Lille     Montpellier   D       1   
2      F1    23/09/14                  Reims       Marseille   A       0   
3      F1    24/09/14                 Bastia          Nantes   D       1   
4      F1    24/09/14  Evian Thonon Gaillard            Lens   H       2   
...    ..         ...                    ...             ...  ..     ...   
21279  I2  26/01/2020                 Ascoli       Frosinone   A       0   
21280  I2  27/01/2020                Perugia         Livorno   H       2   
21281  I2  31/01/2020              Cremonese            Pisa   A       0   
21282  I2  01/02/2020              Frosinone  Virtus Entella   H       2   
21283  I2  01/02/2020                Livorno          Ascoli   A       0   

            AAG       AAS      AASO       AAF       AAC       AAY       AAR  \
0      0

In [5]:
df

,Div,Date,HomeTeam,AwayTeam,FTR,ID_FTR,AAG,AAS,AASO,AAF,AAC,AAY,AAR,HAG,HAS,HASO,HAF,HAC,HAY,HAR
0,F1,20/09/14,Nantes,Nice,H,2,0.182197,0.611454,0.207070,0.320002,2.170601,-0.293938,-0.424521,-1.091481,0.661262,0.467186,0.639428,-0.292570,-0.596982,-0.423557
1,F1,21/09/14,Lille,Montpellier,D,1,-0.672997,0.415223,-0.180212,0.634995,0.251306,-0.293938,-0.424521,-0.238354,0.514423,0.467186,0.639428,1.079451,-0.596982,-0.423557
2,F1,23/09/14,Reims,Marseille,A,0,1.037391,1.396378,1.239821,1.002486,0.525491,-0.293938,-0.424521,-0.238354,-0.121878,-0.048226,0.324357,-0.841378,-0.904258,-0.423557
3,F1,24/09/14,Bastia,Nantes,D,1,-0.672997,0.905800,0.465257,0.949987,0.251306,-0.600178,-0.424521,0.188210,0.024961,0.080627,1.217058,-0.841378,0.939397,-0.423557
4,F1,24/09/14,Evian Thonon Gaillard,Lens,H,2,-0.245400,0.513338,0.465257,0.844990,0.799676,1.237263,-0.424521,-1.518045,-0.023986,-0.692491,0.849475,0.256239,0.324846,-0.423557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21279,I2,26/01/2020,Ascoli,Frosinone,A,0,0.609794,0.807685,0.723445,1.422476,0.799676,2.155983,1.770163,-0.664917,0.367584,0.338333,1.742176,0.256239,2.783053,-0.423557
21280,I2,27/01/2020,Perugia,Livorno,H,2,-1.100595,0.022761,0.077976,0.634995,0.799676,0.931023,1.770163,-0.664917,-0.072932,0.338333,1.374593,0.530643,1.553949,1.775221
21281,I2,31/01/2020,Cremonese,Pisa,A,0,-0.672997,0.268050,0.207070,1.107484,0.799676,1.543503,3.964847,-0.238354,0.122853,-0.177079,0.796964,-0.292570,0.632122,-0.423557
21282,I2,01/02/2020,Frosinone,Virtus Entella,H,2,1.892586,0.562396,0.723445,1.684970,0.799676,0.931023,-0.424521,0.188210,1.101778,1.111450,0.744452,-0.292570,2.168501,-0.423557


In [6]:
x_test = features.iloc[3550:].values
y_test = target.iloc[3550:].values

x_train = features.iloc[:3550].values
y_train = target.iloc[:3550].values

In [7]:
x_train = x_train.astype(float)

In [8]:
y_train = np.asarray(list(y_train))
y_test = np.asarray(list(y_test))

In [9]:
y_train.shape

(3550, 3)

In [10]:
model = create_model()

checkpoint_path = "models/model1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model.fit(x_train,y_train, epochs=20,validation_data=(x_test,y_test),
          callbacks=[cp_callback])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 3550 samples, validate on 17734 samples
Epoch 1/20
3488/3550 [============================>.] - ETA: 0s - loss: 1.0906 - accuracy: 0.6626 - precision: 0.4710 - recall: 0.1001 - auc: 0.5867
Epoch 00001: saving model to models/model1.ckpt
3550/3550 [==============================] - 2s 686us/sample - loss: 1.0910 - accuracy: 0.6624 - precision: 0.4699 - recall: 0.0989 - auc: 0.5858 - val_loss: 1.0856 - val_accuracy: 0.6619 - val_precision: 0.4511 - val_recall: 0.0663 - val_auc: 0.5807
Epoch 2/20
3040/3550 [========================>.....] - ETA: 0s - loss: 1.0714 - accuracy: 0.6644 - precision: 0.4811 - recall: 0.0878 - auc: 0.6006
Epoch 00002: saving model to models/model1.ckpt
3550/3550 [==============================] - 1s 354us/sample - loss: 1.0730 - accuracy: 0.6641 - precision: 0.4776 - recall: 0.0811 - auc: 0.5994 - val_loss: 1.0825 - val_accuracy: 0.6641 

Epoch 19/20
3104/3550 [=========================>....] - ETA: 0s - loss: 1.0385 - accuracy: 0.6807 - precision: 0.5988 - recall: 0.1279 - auc: 0.6471
Epoch 00019: saving model to models/model1.ckpt
3550/3550 [==============================] - 1s 364us/sample - loss: 1.0394 - accuracy: 0.6803 - precision: 0.5926 - recall: 0.1307 - auc: 0.6450 - val_loss: 1.0883 - val_accuracy: 0.6574 - val_precision: 0.4436 - val_recall: 0.1098 - val_auc: 0.5836
Epoch 20/20
3520/3550 [============================>.] - ETA: 0s - loss: 1.0393 - accuracy: 0.6812 - precision: 0.5889 - recall: 0.1440 - auc: 0.6452
Epoch 00020: saving model to models/model1.ckpt
3550/3550 [==============================] - 1s 336us/sample - loss: 1.0391 - accuracy: 0.6812 - precision: 0.5888 - recall: 0.1448 - auc: 0.6454 - val_loss: 1.0881 - val_accuracy: 0.6588 - val_precision: 0.4449 - val_recall: 0.0946 - val_auc: 0.5819


In [15]:
index = np.random.randint(0,107)
print(y_test[index])
print_logits(model.predict(x = x_test[index].reshape(1,14), batch_size = 1),1,3)

[0 0 1]
------------ 0 ------------
prédiction :  0 pourcentage de certitude : 41.6  %
prédiction :  1 pourcentage de certitude : 32.1  %
prédiction :  2 pourcentage de certitude : 26.3  %


In [24]:
y_pred = model.predict(x = x_train)
y_pred = np.argmax(y_pred, axis=1)
conf_mat = tf.math.confusion_matrix(y_train, y_pred)